### 1. 데이터 불러오기

In [21]:
# 라이브러리 정의 import tensorflow.keras
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd

In [22]:
featuresData = pd.read_csv("https://raw.githubusercontent.com/hyokwan/python-lecture/master/dataset/cars.csv")

In [23]:
featuresData.head()

,age,gender,miles,debt,income,sales
0,28,0,23,0,4099,620
1,26,0,27,0,2677,1792
2,30,1,58,41576,6215,27754
3,26,1,25,43172,7626,28256
4,20,1,17,6979,8071,4438


### 2. 특성선정 및 데이터 분리

In [24]:
label = ['sales']
corrdf = featuresData.corr()

In [25]:
features = list(corrdf[(abs(corrdf.sales) > 0.5) & (corrdf.sales <1)].index)
features

['miles', 'debt', 'income']

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
trainingData_features,testData_features,trainingData_label,testData_label,trainingData_all,testData_all = \
train_test_split(featuresData[features],featuresData[label],featuresData, test_size=0.2, random_state=1)

In [39]:
def norm(x):
    return (x/x.max())

trainingData_features_normed = norm(trainingData_features)
testData_features_normed = norm(testData_features)

### 3. 모델 생성

In [44]:
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import tree
from sklearn import linear_model
from sklearn import ensemble
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn import neighbors

In [29]:
inputDim = trainingData_features_normed.loc[0,:].shape
inputDim

(3,)

In [45]:
model_method_svm = svm.SVC(random_state=1)
model_method_dt = tree.DecisionTreeClassifier(random_state =1)    #DecisionTreeRegressor(random_state =1)
model_method_kn = neighbors.KNeighborsClassifier(n_neighbors=10)
model_method_rf = ensemble.RandomForestClassifier(random_state =1, n_estimators= 10)

In [30]:
model = Sequential()
model.add(Dense(units=8, activation="relu",input_shape=inputDim))
model.add(Dense(units=8, activation="relu"))
model.add(Dense(units=1, activation="relu"))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 8)                 32        
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 9         
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________


### 4. 모델 컴파일(훈련준비)

In [31]:
model.compile(loss='mean_squared_error', optimizer = 'adam', metrics=['mean_squared_error','mean_absolute_error'])

### 5. 모델 훈련(callback 정의)

In [32]:
#callback List를 만들기 위한 선언 및 변수 설정
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from datetime import datetime
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from datetime import datetime
import os

log_folder = datetime.now().strftime("%Y%m%d_%H%M%S")



log_dirs = os.path.join("logs",log_folder)

In [33]:
callback_list = [
    TensorBoard(log_dir = log_dirs),
    EarlyStopping(monitor="val_loss", patience=10), 
    ModelCheckpoint(filepath="./model_r_weights.h5", monitor="val_loss", verbose=1, save_best_only=True)   
]

In [47]:
model_svm = model_method_svm.fit(trainingData_features_normed ,trainingData_label)
model_dt  = model_method_dt.fit (trainingData_features_normed ,trainingData_label)
model_kn  = model_method_kn.fit (trainingData_features_normed ,trainingData_label)
model_rf  = model_method_rf.fit (trainingData_features_normed ,trainingData_label)

C:\Users\User\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a

In [34]:
model.fit(x=trainingData_features_normed,
          y=trainingData_label,
          batch_size=32,
          epochs=2000,
          validation_split=0.2,
          callbacks=callback_list)

W1015 15:46:32.096454  7224 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


Train on 616 samples, validate on 154 samples
Epoch 1/2000
224/616 [=========>....................] - ETA: 0s - loss: 210841044.5714 - mean_squared_error: 210841056.0000 - mean_absolute_error: 11603.0635
Epoch 00001: val_loss improved from inf to 187150915.53247, saving model to ./model_r_weights.h5
616/616 [==============================] - 0s 326us/sample - loss: 219879024.8312 - mean_squared_error: 219879008.0000 - mean_absolute_error: 11734.6875 - val_loss: 187150915.5325 - val_mean_squared_error: 187150912.0000 - val_mean_absolute_error: 10914.8682
Epoch 2/2000
 32/616 [>.............................] - ETA: 0s - loss: 209860928.0000 - mean_squared_error: 209860928.0000 - mean_absolute_error: 10949.3545
Epoch 00002: val_loss improved from 187150915.53247 to 187146834.28571, saving model to ./model_r_weights.h5
616/616 [==============================] - 0s 81us/sample - loss: 219874756.7792 - mean_squared_error: 219874752.0000 - mean_absolute_error: 11734.5283 - val_loss: 187146834

Epoch 17/2000
 32/616 [>.............................] - ETA: 0s - loss: 202776032.0000 - mean_squared_error: 202776032.0000 - mean_absolute_error: 11029.2217
Epoch 00017: val_loss improved from 186925629.92208 to 186886685.09091, saving model to ./model_r_weights.h5
616/616 [==============================] - 0s 76us/sample - loss: 219605019.0130 - mean_squared_error: 219605008.0000 - mean_absolute_error: 11724.6973 - val_loss: 186886685.0909 - val_mean_squared_error: 186886672.0000 - val_mean_absolute_error: 10904.3486
Epoch 18/2000
 32/616 [>.............................] - ETA: 0s - loss: 217104608.0000 - mean_squared_error: 217104608.0000 - mean_absolute_error: 11867.0986
Epoch 00018: val_loss improved from 186886685.09091 to 186842632.93506, saving model to ./model_r_weights.h5
616/616 [==============================] - 0s 71us/sample - loss: 219558964.1558 - mean_squared_error: 219558960.0000 - mean_absolute_error: 11723.0107 - val_loss: 186842632.9351 - val_mean_squared_error: 1

Epoch 33/2000
 32/616 [>.............................] - ETA: 0s - loss: 190919232.0000 - mean_squared_error: 190919232.0000 - mean_absolute_error: 10758.4355
Epoch 00033: val_loss improved from 185688019.74026 to 185561022.12987, saving model to ./model_r_weights.h5
616/616 [==============================] - 0s 62us/sample - loss: 218165389.7143 - mean_squared_error: 218165392.0000 - mean_absolute_error: 11672.7754 - val_loss: 185561022.1299 - val_mean_squared_error: 185561040.0000 - val_mean_absolute_error: 10852.0908
Epoch 34/2000
 32/616 [>.............................] - ETA: 0s - loss: 219199552.0000 - mean_squared_error: 219199552.0000 - mean_absolute_error: 11709.8789
Epoch 00034: val_loss improved from 185561022.12987 to 185424915.11688, saving model to ./model_r_weights.h5
616/616 [==============================] - 0s 66us/sample - loss: 218015848.5195 - mean_squared_error: 218015856.0000 - mean_absolute_error: 11667.4609 - val_loss: 185424915.1169 - val_mean_squared_error: 1

Epoch 49/2000
 32/616 [>.............................] - ETA: 0s - loss: 142207936.0000 - mean_squared_error: 142207936.0000 - mean_absolute_error: 9348.0117
Epoch 00049: val_loss improved from 182771532.25974 to 182524348.46753, saving model to ./model_r_weights.h5
616/616 [==============================] - 0s 71us/sample - loss: 214802500.3636 - mean_squared_error: 214802480.0000 - mean_absolute_error: 11551.4932 - val_loss: 182524348.4675 - val_mean_squared_error: 182524352.0000 - val_mean_absolute_error: 10731.9395
Epoch 50/2000
 32/616 [>.............................] - ETA: 0s - loss: 227026992.0000 - mean_squared_error: 227026992.0000 - mean_absolute_error: 12096.5781
Epoch 00050: val_loss improved from 182524348.46753 to 182266416.62338, saving model to ./model_r_weights.h5
616/616 [==============================] - 0s 57us/sample - loss: 214521268.3636 - mean_squared_error: 214521280.0000 - mean_absolute_error: 11541.1699 - val_loss: 182266416.6234 - val_mean_squared_error: 18

...

Epoch 641/2000
 32/616 [>.............................] - ETA: 0s - loss: 17800132.0000 - mean_squared_error: 17800132.0000 - mean_absolute_error: 3408.3499
Epoch 00641: val_loss improved from 15799708.12987 to 15785967.45455, saving model to ./model_r_weights.h5
616/616 [==============================] - 0s 66us/sample - loss: 15713582.8052 - mean_squared_error: 15713584.0000 - mean_absolute_error: 3143.2810 - val_loss: 15785967.4545 - val_mean_squared_error: 15785967.0000 - val_mean_absolute_error: 3114.4607
Epoch 642/2000
 32/616 [>.............................] - ETA: 0s - loss: 16745157.0000 - mean_squared_error: 16745157.0000 - mean_absolute_error: 3131.8894
Epoch 00642: val_loss did not improve from 15785967.45455
616/616 [==============================] - 0s 50us/sample - loss: 15706659.9740 - mean_squared_error: 15706661.0000 - mean_absolute_error: 3140.6545 - val_loss: 15789185.3377 - val_mean_squared_error: 15789186.0000 - val_mean_absolute_error: 3113.5862
Epoch 643/2000
 3

 32/616 [>.............................] - ETA: 0s - loss: 14729025.0000 - mean_squared_error: 14729025.0000 - mean_absolute_error: 2983.0591
Epoch 00673: val_loss improved from 15619214.50649 to 15618258.23377, saving model to ./model_r_weights.h5
616/616 [==============================] - 0s 68us/sample - loss: 15476412.2857 - mean_squared_error: 15476410.0000 - mean_absolute_error: 3089.1328 - val_loss: 15618258.2338 - val_mean_squared_error: 15618257.0000 - val_mean_absolute_error: 3065.1677
Epoch 674/2000
 32/616 [>.............................] - ETA: 0s - loss: 14251334.0000 - mean_squared_error: 14251334.0000 - mean_absolute_error: 3180.0078
Epoch 00674: val_loss improved from 15618258.23377 to 15606917.68831, saving model to ./model_r_weights.h5
616/616 [==============================] - 0s 66us/sample - loss: 15468740.9351 - mean_squared_error: 15468742.0000 - mean_absolute_error: 3088.3391 - val_loss: 15606917.6883 - val_mean_squared_error: 15606917.0000 - val_mean_absolute_

Epoch 690/2000
 32/616 [>.............................] - ETA: 0s - loss: 16173788.0000 - mean_squared_error: 16173788.0000 - mean_absolute_error: 3400.3496
Epoch 00690: val_loss improved from 15559934.10390 to 15557139.81818, saving model to ./model_r_weights.h5
616/616 [==============================] - 0s 73us/sample - loss: 15386677.8506 - mean_squared_error: 15386679.0000 - mean_absolute_error: 3066.3286 - val_loss: 15557139.8182 - val_mean_squared_error: 15557140.0000 - val_mean_absolute_error: 3044.5986
Epoch 691/2000
 32/616 [>.............................] - ETA: 0s - loss: 21313336.0000 - mean_squared_error: 21313336.0000 - mean_absolute_error: 4014.3408
Epoch 00691: val_loss improved from 15557139.81818 to 15555292.66234, saving model to ./model_r_weights.h5
616/616 [==============================] - 0s 63us/sample - loss: 15382264.6883 - mean_squared_error: 15382264.0000 - mean_absolute_error: 3064.9907 - val_loss: 15555292.6623 - val_mean_squared_error: 15555293.0000 - val

 32/616 [>.............................] - ETA: 0s - loss: 17343196.0000 - mean_squared_error: 17343196.0000 - mean_absolute_error: 3364.3457
Epoch 00723: val_loss improved from 15468916.70130 to 15464615.09091, saving model to ./model_r_weights.h5
616/616 [==============================] - 0s 75us/sample - loss: 15260124.8182 - mean_squared_error: 15260125.0000 - mean_absolute_error: 3032.0408 - val_loss: 15464615.0909 - val_mean_squared_error: 15464614.0000 - val_mean_absolute_error: 3014.3928
Epoch 724/2000
 32/616 [>.............................] - ETA: 0s - loss: 12477704.0000 - mean_squared_error: 12477704.0000 - mean_absolute_error: 2981.6467
Epoch 00724: val_loss improved from 15464615.09091 to 15462514.80519, saving model to ./model_r_weights.h5
616/616 [==============================] - 0s 70us/sample - loss: 15257743.7662 - mean_squared_error: 15257745.0000 - mean_absolute_error: 3032.1282 - val_loss: 15462514.8052 - val_mean_squared_error: 15462515.0000 - val_mean_absolute_

Epoch 740/2000
 32/616 [>.............................] - ETA: 0s - loss: 17334792.0000 - mean_squared_error: 17334792.0000 - mean_absolute_error: 3052.4102
Epoch 00740: val_loss did not improve from 15416860.96104
616/616 [==============================] - 0s 55us/sample - loss: 15211419.4286 - mean_squared_error: 15211420.0000 - mean_absolute_error: 3019.5071 - val_loss: 15429847.8701 - val_mean_squared_error: 15429846.0000 - val_mean_absolute_error: 3003.9700
Epoch 741/2000
 32/616 [>.............................] - ETA: 0s - loss: 14075985.0000 - mean_squared_error: 14075985.0000 - mean_absolute_error: 2992.6479
Epoch 00741: val_loss did not improve from 15416860.96104
616/616 [==============================] - 0s 44us/sample - loss: 15207035.8442 - mean_squared_error: 15207035.0000 - mean_absolute_error: 3018.7642 - val_loss: 15432906.8182 - val_mean_squared_error: 15432907.0000 - val_mean_absolute_error: 3003.5295
Epoch 742/2000
 32/616 [>.............................] - ETA: 0s 

### 6. 모델 추론(validation)

In [49]:
predict_svm = model_svm.predict(testData_features_normed)
predict_dt  = model_dt.predict(testData_features_normed)
predict_kn  = model_kn.predict(testData_features_normed)
predict_rf  = model_rf.predict(testData_features_normed)

In [50]:
predictDf_svm = pd.DataFrame(predict_svm, columns=["predict_svc"])
predictDf_dt = pd.DataFrame(predict_dt, columns=["predict_dt"])
predictDf_kn = pd.DataFrame(predict_kn, columns=["predict_kn"])
predictDf_rf = pd.DataFrame(predict_rf, columns=["predict_rf"])

In [69]:
predictDf_dnn = pd.DataFrame(model.predict(testData_features_normed), columns=["predict_dnn"])

W1015 16:07:38.085831  7224 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


TypeError: only size-1 arrays can be converted to Python scalars

In [40]:
model.evaluate(x=testData_features_normed,
              y=testData_label)

##nored 전 : [233259838.34196892, 233259860.0, 12164.15]
##nored 후 : [13376948.481865285, 13376950.0, 2895.6824]


W1015 15:48:43.150325  7224 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


193/193 [==============================] - 0s 31us/sample - loss: 13376948.4819 - mean_squared_error: 13376950.0000 - mean_absolute_error: 2895.6824


[13376948.481865285, 13376950.0, 2895.6824]

In [41]:
lose, mse, mae = model.evaluate(x=testData_features_normed,
              y=testData_label)

W1015 15:49:22.843355  7224 training.py:504] Falling back from v2 loop because of error: Failed to find data adapter that can handle input: <class 'pandas.core.frame.DataFrame'>, <class 'NoneType'>


193/193 [==============================] - 0s 31us/sample - loss: 13376948.4819 - mean_squared_error: 13376950.0000 - mean_absolute_error: 2895.6824


In [61]:
len(predictDf_dnn)

193

In [58]:
testData_freatures2 = testData_features.reset_index(drop=True)
testData_label2 = testData_label.reset_index(drop=True)

In [81]:
finalData = pd.concat([testData_freatures2,testData_label2,predictDf_dnn,predictDf_svm,predictDf_dt,predictDf_kn,predictDf_rf],axis=1)
finalData.head(10)

,miles,debt,income,sales,predict_dnn,predict_svc,predict_dt,predict_kn,predict_rf
0,20,414,9900,5265,9757.305664,17677,5029,1376,6041
1,32,51908,7169,23894,24227.353516,25125,25118,22508,25715
2,32,9503,5479,9319,10429.889648,7596,9533,6042,8526
3,23,5809,5548,13722,8390.822266,7596,13790,1285,13790
4,29,15752,6277,22145,12680.096680,7596,12256,6042,6626
5,12,903,6314,2440,6303.268555,3510,4101,2228,3779
6,30,0,1673,2182,4319.287109,2869,1789,763,1789
7,35,4339,7852,6025,10967.161133,17677,6250,6240,16678
8,25,4133,4212,4376,7045.677246,4609,1836,1492,3978
9,25,5844,5376,12748,8478.010742,7596,6376,1285,6376


In [82]:
finalData.dtypes

miles            int64
debt             int64
income           int64
sales            int64
predict_dnn    float32
predict_svc      int64
predict_dt       int64
predict_kn       int64
predict_rf       int64
dtype: object

In [77]:
finalData["predictDf_dnn"] = finalData.predict_dnn.astype(int)

In [78]:
from sklearn.metrics import mean_absolute_error

In [84]:
#결과 테스트하기
ac_score_dnn = mean_absolute_error(finalData.sales, finalData.predict_dnn)
ac_score_svc = mean_absolute_error(finalData.sales, finalData.predict_svc)
ac_score_dt = mean_absolute_error(finalData.sales, finalData.predict_dt)
ac_score_kn = mean_absolute_error(finalData.sales, finalData.predict_kn)
ac_score_rf = mean_absolute_error(finalData.sales, finalData.predict_rf)

##결과 리포트하기
print("Accuracy dnn =", ac_score_dnn)
print("Accuracy svm =", ac_score_svc)
print("Accuracy dt=", ac_score_dt)
print("Accuracy kn=", ac_score_kn)
print("Accuracy rf=", ac_score_rf)

Accuracy dnn = 2895.682270583711
Accuracy svm = 3676.958549222798
Accuracy dt= 2848.1502590673576
Accuracy kn= 4532.430051813471
Accuracy rf= 3395.481865284974
